In [45]:
import pandas as pd
import ast
import csv

In [2]:
## Load in dataframes for the process

matches_df = pd.read_csv('data/okcupid_matches.csv') 
matches_df = matches_df.rename(columns={'Unnamed: 0': 'Person_id'})

bios_text = pd.read_csv('data/okcupid_profiles.csv')
bios_text.reset_index(inplace=True)
bios_text = bios_text.rename(columns={'index': 'Person_id'})

## Joining the bios together

bios_text.fillna(' ', inplace=True)
bios_text['allessays'] = bios_text[['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8']].apply(lambda x: ' '.join(x), axis=1)
bios_text = bios_text[['Person_id', 'allessays']]

## Join two df together on person_id

finetune_df = pd.merge(matches_df, bios_text, on='Person_id')

In [39]:
## Pulling matches with high cosine similarity

input_output_pairs = []

for index, row in finetune_df.iterrows():
    if row['matches'] != '[]':
       for match in ast.literal_eval(row['matches']):
            if match[1] >= .81 and match[1] <= .99:
                match_id = match[0]
                matching_row = finetune_df[finetune_df['Person_id'] == match_id]
                input_output_pairs.append((row['allessays'], matching_row['allessays'].iloc[0]))

In [43]:
unique_pairs = set()

for tup in input_output_pairs:
    sorted_tup = tuple(sorted(set(tup)))
    unique_pairs.add(sorted_tup)

unique_pairs_list = list(unique_pairs)

print(unique_pairs_list)

[("i feel very content and in a way lucky to have moved to san francisco; and loving every second of it. actually, nut sure what took me so long to get here! some of my favorite activities so far: biking to sausalito, enjoying one of the cafes and riding on exploring the beautiful surroundings, going skiing to tahoe (i worked as a ski instructor in canada for a season), taking weekend trips to the beautiful places surrounding sf, exploring the city and all the wonderful neighborhoods, parks and farmers markets, getting out and enjoying the cultural activities and live music that the city has to offer. prior to the move, i took some time off, and went on a 5 month trip to europe. i like to take the more european approach to life, a good coffee over an interesting conversation or a book sitting outside is up there as one of my favorite ways to spend an hour or two. very important for me to live a balanced life, not to take things too seriously and most importantly to keep life interestin

In [46]:
csv_file = "data/top_pairs.csv"

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(unique_pairs_list[:10])
